In [4]:
import time
import torch.optim
import torch.nn as nn
from torchvision import models
from torch.autograd import Variable
from datasets.dataset import train_loader
import matplotlib.pyplot as plt

C:\Users\Valdone\Desktop\uni\6.semester\KI\cnn-fine-grained-image-recognition\datasets\dataset.py:39: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  x.append(f[name][file].value)
Matplotlib is building the font cache; this may take a moment.


In [5]:
learning_rate = 1e-3
input_channels = 1
output_features = 6
epoch = 1
save_model_name = 'convolutional.pth'

In [6]:
conv = torch.hub.load('pytorch/vision:v0.9.0', 'vgg19', pretrained=True)
conv.eval()
conv.conv1 = nn.Conv2d(input_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
conv.fc = nn.Linear(in_features=512, out_features=output_features, bias=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(conv.parameters(), lr=learning_rate)

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to C:\Users\Valdone/.cache\torch\hub\v0.9.0.zip
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\Valdone/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth


In [7]:
epoch_data = list()

for epoch_number in range(epoch):
    running_loss, count, acc = 0., 0, 0.
    print(time.asctime())
    for img, label in train_loader:
        img = Variable(img)
        label = Variable(label)
        output = conv(img)
        optimizer.zero_grad()
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        acc += (torch.max(output, dim=1)[1]==label).sum()
        count += img.size(0)
    print(epoch_number, count, running_loss, int(acc)/count)
    epoch_data.append((count, running_loss, int(acc)/count))
print(time.asctime())

Wed May 12 20:07:55 2021


KeyboardInterrupt: 

In [ ]:
epoch_loss = list()

In [ ]:
epoch_loss.append((4,6,8))

In [ ]:
plt.plot(range(epoch), epoch_loss)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
torch.save(conv, save_model_name)

In [ ]:
count, acc = 0, 0.
for data in test_loader:
    img, label = data
    img = Variable(img).cuda()
    label = Variable(label).cuda()
    output = conv(img)
    acc += (torch.max(output, dim=1)[1] == label).sum()
    count += img.size(0)
print(count, running_loss, int(acc)/count)